# Learning the basics of algo trading with Oanda

In [2]:
# imports
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [20]:
# load environment variables
load_dotenv()

OANDA_API_KEY = os.getenv("OANDA_API_KEY")
OANDA_ACCT_NUM = os.getenv("OANDA_ACCT_NUM")
OANDA_URL = os.getenv("OANDA_URL")
SECURE_HEADER = {
    "Authorization": f"Bearer {OANDA_API_KEY}",
}

In [21]:
# create session
session = requests.Session()

In [4]:
# get instruments
instruments_df = pd.read_pickle("instruments.pkl") 

In [24]:
def fetch_candles(pair_name, count, granularity):
   url = f"{OANDA_URL}/v3/instruments/{pair_name}/candles"
   params = {
       "count": count,
       "granularity": granularity,
       "price": "MBA"
   }
   response = session.get(url, headers=SECURE_HEADER, params=params)
   return response.status_code, response.json()

In [25]:
# test
code, data = fetch_candles('EUR_USD', 10, 'H1')

In [ ]:
data

In [29]:
# build a dataframe
def get_candles_df(data):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    our_data = []
    for candle in data["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

In [ ]:
get_candles_df(data)

In [34]:
def save_file(candles_df, pair, count, granularity) :
    candles_df.to_pickle(f"historical_data/{pair}_{granularity}_{count}.pkl")

In [32]:
def create_data(pair, count, granularity):
    res_code, res_data = fetch_candles(pair, count, granularity)
    if res_code != 200:
        print(f"Error fetching candles for {pair}")
        return
    candles_df = get_candles_df(res_data)
    print (f"{pair} loaded {candles_df.shape[0]} candles from {candles_df.time.min()} to {candles_df.time.max()}")
    save_file(candles_df, pair, count, granularity)


In [35]:
# currencies intersted in trading
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD']
# test currencies
for p1 in our_curr:
    for p2 in our_curr:
        pair = f"{p1}_{p2}"
        if pair in instruments_df.name.unique():
            create_data(pair, 4000, 'H1')

EUR_USD loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
EUR_GBP loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
EUR_JPY loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
EUR_CHF loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
EUR_NZD loaded 3999 candles from 2023-01-15T22:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
EUR_CAD loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
USD_JPY loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
USD_CHF loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
USD_CAD loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
GBP_USD loaded 3999 candles from 2023-01-15T23:00:00.000000000Z to 2023-09-05T12:00:00.000000000Z
GBP_JPY loaded 3999 

In [ ]:
instruments_df.info()

In [8]:
instruments_df.columns

Index(['name', 'type', 'display_name', 'pip_location', 'margin_rate'], dtype='object')

In [ ]:
instruments_df.name.unique()